In [1]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import pandas as pd

In [2]:
driver = webdriver.Chrome()

In [3]:
def get_raw_data(url):
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "live-table")))

    while True:
        # Find all current "Show more" links
        show_more_links = driver.find_elements(By.XPATH, "//span[contains(text(), 'Show more matches')]/parent::a")
        if not show_more_links:
            break  # No more links to click

        for link in show_more_links:
            try:
                # Scroll to the link to ensure it's visible
                driver.execute_script("arguments[0].scrollIntoView();", link)
                link.click()
                # Wait for the link to become stale (content expanded)
                WebDriverWait(driver, 5).until(EC.staleness_of(link))
            except Exception:
                pass  # Ignore if already expanded or error

    soup = bs(driver.page_source, 'html.parser')
    return soup

In [4]:
soup = get_raw_data('https://www.livesport.com/uk/football/england/premier-league-2023-2024/results/')

In [5]:
events = soup.find_all('div', class_='event__match event__match--withRowLink event__match--static event__match--twoLine')
last_event = soup.find_all('div', class_='event__match event__match--withRowLink event__match--static event__match--last event__match--twoLine')

In [6]:
events[-1].find('div', class_='event__time')

<div class="event__time">12.08. 12:30</div>

In [7]:
len(last_event)

44

* The match fixtures are structured in rounds, as if they were individual tables separating the first to the last event of each round
* You can use the events variable to get the regular events and the last_event to get the last event of each round respectively
* However, despite the data exploration, there seems to be a mismatch between the length of last event and number of rounds. Which is left to be solved
* Nonetheless, some progress was made.

# <h4>Fix the `last_event` length mismatch and craft a function to fully retrieve data</h4>

In [8]:
last_event[-1].find('div', class_='event__time')

<div class="event__time">11.08. 20:00</div>

In [9]:
ex = soup.find('div', class_='sportName soccer')

In [10]:
all_div = []
for div in ex.find_all('div', recursive=False):
    classes = div.get('class')
    if not 'event__match' in classes:
        continue 
    all_div.append(div)

In [11]:
classes

['event__match',
 'event__match--withRowLink',
 'event__match--static',
 'event__match--last',
 'event__match--twoLine']

In [12]:
all_div[-1].find('div', class_='wcl-participant_bctDY event__homeParticipant').text, all_div[-1].find('div', class_='wcl-participant_bctDY event__awayParticipant').text

('Burnley', 'Man City')

In [13]:
all_div[-1].find('div', class_='wcl-participant_bctDY event__homeParticipant').text

'Burnley'

In [14]:
dates = []
for match in all_div:
    dates.append(match.find('div', class_='event__time').text)

dates

['19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '19.05. 16:00',
 '15.05. 20:00',
 '15.05. 19:45',
 '14.05. 20:00',
 '13.05. 20:00',
 '12.05. 16:30',
 '11.05. 17:30',
 '11.05. 15:00',
 '11.05. 15:00',
 '11.05. 15:00',
 '11.05. 15:00',
 '11.05. 15:00',
 '11.05. 15:00',
 '11.05. 12:30',
 '06.05. 20:00',
 '05.05. 16:30',
 '05.05. 14:00',
 '05.05. 14:00',
 '04.05. 17:30',
 '04.05. 15:00',
 '04.05. 15:00',
 '04.05. 15:00',
 '04.05. 12:30',
 '03.05. 20:00',
 '02.05. 19:30',
 '28.04. 16:30',
 '28.04. 14:00',
 '28.04. 14:00',
 '27.04. 20:00',
 '27.04. 17:30',
 '27.04. 15:00',
 '27.04. 15:00',
 '27.04. 15:00',
 '27.04. 15:00',
 '27.04. 12:30',
 '25.04. 20:00',
 '24.04. 20:00',
 '24.04. 20:00',
 '24.04. 20:00',
 '24.04. 19:45',
 '23.04. 20:00',
 '21.04. 16:30',
 '21.04. 15:00',
 '21.04. 15:00',
 '21.04. 13:30',
 '20.04. 19:30',
 '20.04. 15:00',
 '20.04. 15:00',
 '15.04. 20:00',
 '14.04. 16:30

In [ ]:
def get_matches():
    # soup = get_raw_data('https://www.livesport.com/uk/football/england/premier-league-2023-2024/results/')
    fixtures = soup.find('div', class_='sportName soccer')
    all_div = []
    for div in fixtures.find_all('div', recursive=False):
        classes = div.get('class')
        if not 'event__match' in classes:
            continue
        all_div.append(div)

    first_year = 2023
    second_year = 2024

    season = []
    dates = []
    for match in all_div:
        home_team = match.find('div', class_='wcl-participant_bctDY event__homeParticipant').text
        away_team = match.find('div', class_='wcl-participant_bctDY event__awayParticipant').text
        date_el = match.find('div', class_='event__time')
        date = date_el.get_text()
        home_score = match.find('span', class_='wcl-matchRowScore_fWR-Z wcl-isFinal_7U4ca event__score event__score--home').text
        away_score = match.find('span', class_='wcl-matchRowScore_fWR-Z wcl-isFinal_7U4ca event__score event__score--away').text

        
        day_month = date.split()[0].replace('.', '')
        time_part = date.split()[1]
        
        day = int(day_month[:2])
        month = int(day_month[2:])
        year = first_year if month >= 8 else second_year

        match_day = f"{year}-{month:02d}-{day:02d}"
            

        dates.append(date)
        season.append({
            'home_team': home_team,
            'away_team': away_team,
            'match_day': match_day,
            'time': time_part,
            'home_score': home_score,
            'away_score': away_score
        })

    return pd.DataFrame(season), dates

In [34]:
df, dates = get_matches()
# df.to_csv('epl_fixtures_2023_2024.csv', index=False)

In [35]:
df

,home_team,away_team,match_day,time,home_score,away_score
0,Arsenal,Everton,2024-05-19,16:00,2,1
1,Brentford,Newcastle,2024-05-19,16:00,2,4
2,Brighton,Man Utd,2024-05-19,16:00,0,2
3,Burnley,Nottm Forest,2024-05-19,16:00,1,2
4,Chelsea,Bournemouth,2024-05-19,16:00,2,1
...,...,...,...,...,...,...
375,Brighton,Luton,2023-08-12,15:00,4,1
376,Everton,Fulham,2023-08-12,15:00,0,1
377,Sheff Utd,Crystal Palace,2023-08-12,15:00,0,1
378,Arsenal,Nottm Forest,2023-08-12,12:30,2,1


* I was able to successfully extract the data, however there are some parts that still need attention
* The date column has no year associated

    ## <h4>Points to work on</h4>

* Add year and separate time from dates
* Refactor the function to iterate through all seasons
* Append all the data in one complete spreadsheet

In [31]:
first_year = 2023
second_year = 2024

match_day = []

for date in dates:
    day_month = date.split()[0].replace('.', '')
    time_part = date.split()[1]
    day = int(day_month[:2])
    month = int(day_month[2:])
    year = first_year if month >= 8 else second_year

    match_day.append(f"{year}-{month}-{day}")


In [4]:
def get_epl_fixtures(first_year, second_year):
    soup = get_raw_data(f'https://www.livesport.com/uk/football/england/premier-league-{first_year}-{second_year}/results/')
    fixtures = soup.find('div', class_='sportName soccer')
    all_div = []
    for div in fixtures.find_all('div', recursive=False):
        classes = div.get('class')
        if not 'event__match' in classes:
            continue
        all_div.append(div)

    season = []
    dates = []
    for match in all_div:
        home_team = match.find('div', class_='wcl-participant_bctDY event__homeParticipant').text
        away_team = match.find('div', class_='wcl-participant_bctDY event__awayParticipant').text
        date_el = match.find('div', class_='event__time')
        date = date_el.get_text()
        home_score = match.find('span', class_='wcl-matchRowScore_fWR-Z wcl-isFinal_7U4ca event__score event__score--home').text
        away_score = match.find('span', class_='wcl-matchRowScore_fWR-Z wcl-isFinal_7U4ca event__score event__score--away').text

        
        day_month = date.split()[0].replace('.', '')
        time_part = date.split()[1]
        
        day = int(day_month[:2])
        month = int(day_month[2:])
        year = first_year if month >= 8 else second_year

        match_day = f"{year}-{month:02d}-{day:02d}"
            
        season.append({
            'home_team': home_team,
            'away_team': away_team,
            'match_day': match_day,
            'time': time_part,
            'home_score': home_score,
            'away_score': away_score
        })

    return pd.DataFrame(season)

In [ ]:
initial_year = 2023
final_year = 1901

all_seasons = []
missing_seasons = []

while initial_year > final_year:
    try:
        df = get_epl_fixtures(initial_year, initial_year + 1)
        # Add a season column to identify which season each match belongs to
        df['season'] = f"{initial_year}-{initial_year + 1}"
        all_seasons.append(df)
        print(f"Completed season {initial_year}-{initial_year + 1}")
    except Exception as e:
        missing_seasons.append(f"{initial_year}-{initial_year + 1}")
        print(f"Error processing season {initial_year}-{initial_year + 1}: {e}")
    
    initial_year -= 1

# Concatenate all DataFrames into one
if all_seasons:
    complete_df = pd.concat(all_seasons, ignore_index=True)
    print(f"Total matches collected: {len(complete_df)}")
    
    # Save to CSV
    complete_df.to_csv('epl_all_seasons.csv', index=False)
    print("Data saved to epl_all_seasons.csv")

Completed season 2023-2024
Completed season 2022-2023
Completed season 2021-2022
Completed season 2020-2021
Completed season 2019-2020
Completed season 2018-2019
Completed season 2017-2018
Completed season 2016-2017
Completed season 2015-2016
Completed season 2014-2015
Completed season 2013-2014
Completed season 2012-2013
Completed season 2011-2012
Completed season 2010-2011
Completed season 2009-2010
Completed season 2008-2009
Completed season 2007-2008
Completed season 2006-2007
Completed season 2005-2006
Completed season 2004-2005
Completed season 2003-2004
Completed season 2002-2003
Completed season 2001-2002
Completed season 2000-2001
Completed season 1999-2000
Completed season 1998-1999
Completed season 1997-1998
Completed season 1996-1997
Completed season 1995-1996
Completed season 1994-1995
Completed season 1993-1994
Completed season 1992-1993
Completed season 1991-1992
Completed season 1990-1991
Completed season 1989-1990
Completed season 1988-1989
Completed season 1987-1988
C